In [1]:
from selenium import webdriver
import selenium as se
import pandas as pd
import time
import os

KLUE_HOME_URL = "https://klue.kr/"
LOGIN_INFO ={'ID':'nachiketa3299', 'PW':'Klue12qwaszx!'}
WEBDRIVER_PATH = "./chromedriver"
SCROLL_PAUSE_TIME = 0.5
SAVE_PATH = './result' 

def scroll_down(driver, scroll_pause_time):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return

def scroll_down_half_page(driver, N):
    for i in range(0, N):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    return
    

def login(driver, login_info):
    driver.find_element_by_xpath("/html/body/app-root/menubar/div/ul/menubar-guest/span[2]").click()
    id_input = driver.find_element_by_xpath("/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]")
    pw_input = driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]')
    login_button = driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button')
    id_input.send_keys(LOGIN_INFO['ID'])
    pw_input.send_keys(LOGIN_INFO['PW'])
    login_button.click()
    return

def numbering(string):
    list_string = list(string)
    del list_string[0], list_string[-1]
    return int(''.join(list_string))

def search(driver, query):
    search_bar = driver.find_element_by_class_name('lecture-search-content-context-text')
    search_button = driver.find_element_by_class_name('lecture-search-content-context-btn')
    
    search_bar.clear()
    search_bar.send_keys(query)
    search_button.click()
    driver.implicitly_wait(1)
    return
    

2개의 드라이버(`DRIVER`, `POP`) 실행 및 로그인 후 `DRIVER`는 [검색 페이지](https://klue.kr/lecture/search/)로 이동

In [11]:
DRIVER = webdriver.Chrome(WEBDRIVER_PATH)
POP = webdriver.Chrome(WEBDRIVER_PATH)

assert DRIVER.get(KLUE_HOME_URL)
assert POP.get(KLUE_HOME_URL)


try:
    DRIVER.get(KLUE_HOME_URL)
    
except UnexpectedAlertPresentException:
    DRIVER.switch_to.alert.accept()
    DRIVER.get(KLUE_HOME_URL)
    
try:
    POP.get(KLUE_HOME_URL)

except UnexpectedAlertPresentException:
    POP.switch_to.alert.accept()
    POP.get(KLUE_HOME_URL)

login(DRIVER, LOGIN_INFO)
login(POP, LOGIN_INFO)

DRIVER.get('https://klue.kr/lecture/search/')


##### `result.csv`에서 모든 과목의 학수번호의 앞 4자리를 불러온 후 교양 학수번호와 함께 `course_list`에 저장

In [4]:
df = pd.read_csv('result.csv')
course_list = sorted(list(set(df['학수번호'])))
course_list = sorted(list(set([course[:4] for course in course_list])) + ['IFLS', 'SPFL', 'SPGE', 'GEST', 'GEWR', 'GELA', 'GESO', 'GEFC', 'GEHI', 'GECE', 'GELI', 'GEQR'])

In [200]:
len(course_list)

94

다음은 실제로 크롤링이 일어나는 단계다.
크롤링이 완료된 후에 만들어지는 `DB`리스트의 한 요소인 `course_info`는 다음과 같다.

`course_info = [과목코드, 강의이름, 교수이름, 학기, 이수구분, 학점, [[사용자이름(1), 총평, 출석체크, 난이도, 학습량, 학점, 성취감], [사용자이름(2), 총평, 출석체크, 난이도, 학습량, 학점, 성취감], ...]`

본 크롤러는 __2017년__부터의 강의평가만을 대상으로 한다.

다음과 같은 순서로, `course_list`의 각 항목마다 다음 단계를 실행한다.
>1. 검색어 입력 (과목코드를 입력함)
>2. 웹 페이지가 순차적으로 로딩되도록 페이지를 내리고, 가장 마지막 항목이 2017보다 이전이라면, 스크롤을 중단하고 2017-1학기부터의 아이템들로만 `search_items`를 구성함.
>3. 검색 결과의 각 항목을 크롤링하여, 2017-1학기 이후의 학기만을 대상으로 링크를 취합, `course_info` 리스트 생성
>4. 각 링크들의 페이지에 들어감, 각 링크별로 다음의 단계를 밟음
>>    1. 웹 페이지가 로딩되도록 최 하단까지 스크롤
>>    2. 각 항목을 크롤링하여 구조화함
>>    3. `course_info`에 구조화된 사용자 평가 데이터 취합
>5. 취합된 `course_info`를 `DB`에 취합함.



In [6]:
# 0 ~ 94
START = 17
END = 94
for college_index in range(START, END):
    course = course_list[college_index]
    print("=" * 30, course, "=" * 30)
    
    
    # 1. 검색어 입력 (과목코드를 입력함)
    search(DRIVER, course)
    
    # 2. 웹 페이지가 순차적으로 로딩되도록 페이지를 내리고, 가장 마지막 항목이 2017보다 이전이라면, 스크롤을 중단하고 2017-1학기부터의 아이템들로만 `search_items_links`를 구성함.
    while True:
        search_items = DRIVER.find_elements_by_class_name('lecture-search-result')
        if int(search_items[-1].find_element_by_class_name('lecture-search-result-info').text.split('-')[0]) < 2017:
            break
        else:
            scroll_down_half_page(DRIVER, 20)
    search_items_links = [item.find_elements_by_class_name('lecture-search-result-right-row')[1].find_element_by_tag_name('a').get_attribute('href') for item in search_items if int(item.find_element_by_class_name('lecture-search-result-info').text.split(' ')[0][:4]) >= 2017]
        
    # 3. 검색 결과의 각 항목을 크롤링하여, 2017-1학기 이후의 학기만을 대상으로 링크를 취합, `course_info` 리스트 생성
    link_list = []
    
    INNER_START = 0
    END = 4
    MAX_LEN = len(search_items_links)
    result_string = f'> {course}, course index {college_index:3d}'
    
    for i in range(INNER_START, MAX_LEN):
        
        processed = (i / len(search_items_links)) * 100
        page_id = int(search_items_links[i].split('/')[-1])
        POP.get(search_items_links[i])
        scroll_down(POP, SCROLL_PAUSE_TIME)
        POP.implicitly_wait(10)
        print(f'{result_string}, page id({page_id:5d}) {i + 1:4d} of {MAX_LEN:3d}, (current = {processed:>5.2f}%, total = {(college_index / 94.0) * 100 + (1/94.0) * processed :>7.4f}%)')

        
    
        main_info_container = POP.find_element_by_class_name('lecture-index-main')
        main_info_container_table = POP.find_element_by_class_name('lecture-main-info-table').find_elements_by_tag_name('td')

        _course_code = main_info_container_table[1].text
        _course_title = main_info_container.find_element_by_class_name('lecture-index-main-title').text
        _professor = main_info_container.find_element_by_class_name('lecture-title-professor').text.split(' ')[0]
        _semester = main_info_container_table[0].text
        _course_type = main_info_container_table[2].text
        _course_point = main_info_container_table[4].text[0]

        
        user_ratings = POP.find_elements_by_class_name('lecture-detail-content')
        # [사용자이름(1), 총평, 출석체크, 난이도, 학습량, 학점, 성취감, TEXT]
        user_rating_wrapper = []
        for user_rating in user_ratings:
            try:
                
                sliders = user_rating.find_elements_by_class_name('slider-klue-outer')
                ratings_in_sliders = [str(int(rating.value_of_css_property('width').replace('px', '')) // 26) for rating in sliders]

                _user_name = user_rating.find_element_by_class_name('lecture-eval-content-user').text.replace(' 님의 의견', '')
                _rating_sum = ratings_in_sliders[0]
                _rating_attendance = ratings_in_sliders[1]
                _rating_difficulty = ratings_in_sliders[2]
                _rating_amount = ratings_in_sliders[3]
                _rating_grade = ratings_in_sliders[4]
                _rating_accomplish = ratings_in_sliders[5]
                _rating_text = user_rating.find_element_by_class_name('lecture-eval-content-context').text
            except:
                print('\t no data in this container')
                _user_name = 'NoData'
                _rating_sum = 'NoData'
                _rating_attendance = 'NoData'
                _rating_difficulty = 'NoData'
                _rating_amount = r'NoData'
                _rating_grade = 'NoData'
                _rating_accomplish = 'NoData'
                _rating_text = 'NoData'

            user_rating_detail = [_user_name, _rating_sum, _rating_attendance, _rating_difficulty, _rating_amount, _rating_grade, _rating_accomplish, _rating_text]

            user_rating_wrapper.append(user_rating_detail)
 
        
        # course_info = [과목코드, 강의이름, 교수이름, 학기, 이수구분, 학점, [[사용자이름(1), 총평, 출석체크, 난이도, 학습량, 학점, 성취감], [사용자이름(2), 총평, 출석체크, 난이도, 학습량, 학점, 성취감], ...]
        course_info = [_course_code, _course_title, _professor, _semester, _course_type, _course_point, user_rating_wrapper]

        link_list.append(course_info)

        
    
    DB = []
    for link in link_list:
        for user_rating in link[6]:
            record = link[0:6] + user_rating
            DB.append(record)
            
    
    labels = ['과목코드', '강의이름', '교수이름', '학기구분', '이수구분', '학점', '사용자이름', '총평', '출석체크', '난이도', '학습량', '학점', '성취감', '강의평']    
    database = pd.DataFrame(DB, columns=labels)
    database.to_csv(SAVE_PATH + '/' + course + ".csv", mode='w')
    
        

============================== ECON ==============================


IndexError: list index out of range

In [7]:
search_items

[]

In [6]:
_rating_sum

'3'

In [7]:
ratings_in_sliders

[]